In [1]:
import pandas as pd
import numpy as np
import os
from ydata_profiling import ProfileReport
import utils

In [2]:
with open("bdd_profilweb.csv", "r") as file:
    file_contents = file.read()
file_contents=file_contents.replace('Ã©','é')
file_contents=file_contents.replace('Ã¨','è')
rows=file_contents.split('\n')
header=rows[0].split(';')
data=[i.split(';') for i in rows[1:]]
df=pd.DataFrame(data,columns=header)

In [62]:
df_clean=df.copy()
#df_clean.replace('?',,inplace=True)
df_clean.replace('manquant',np.nan,inplace=True)
df_clean.replace('<18 ans',np.nan,inplace=True)



In [92]:
df_clean[df['activite']=='inactif'][df['utilisation_credit']=='-']['utilisation_credit']='--'
df['C'] = df.apply(lambda x: '--' if x['A'] > x['B'] else x['A'], axis = 1)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1112\284066975.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clean[df['activite']=='inactif'][df['utilisation_credit']=='-']['utilisation_credit']='--'


In [80]:
df[df['tr_age']=='<18 ans']

,num_cli,connexion_mois,source_web,typ_client,montant_credit,activite,anciennete,utilisation_credit,logmt,sitfam,dept,tr_age,revfyr,assurance
25360,25361,2,Liens Sponsorisés,Carte de credit enseigne partenaire,2.]1000e- 3000e],actif,>24 mois,++,Accession pro,Marie - Union libre,Midi-Pyrenees,<18 ans,2000 a 2999,credit assure
36258,36259,11-20,Webmails,Pas client ou ancien client,?,?,?,?,Autres,NR,ile-de-France,<18 ans,manquant,?
42164,42165,5-6,Notoriété,Reserve d'argent,4.]3000e- 6000e],actif,>24 mois,+++,Locataire,Marie - Union libre,Provence-Alpes-Côte-Azur,<18 ans,1500 a 1999,credit non assure


In [3]:
from matplotlib import pyplot as plt

In [111]:
pd.crosstab(df.typ_client,df.montant_credit)

montant_credit,1.<=1000e,2.]1000e- 3000e],4.]3000e- 6000e],5.]6000e- 8000e],6.]8000e- 12000e],7.>12000e,?
typ_client,,,,,,,
Carte de credit enseigne partenaire,3078,5168,281,1,0,0,0
Credit voiture,62,232,827,395,840,739,0
Pas client ou ancien client,0,0,0,0,0,0,3540
Reserve d'argent,4222,10039,13693,3846,2080,957,0


In [4]:
from mca import MCA

In [ ]:
!

In [110]:
import cupy as cp
# Créez un objet GPU
gpu = cp.cuda.Device(0)
# Exécutez un code CUDA sur la GPU<

ModuleNotFoundError: No module named 'cupy'

In [5]:
mcaFic=MCA(pd.get_dummies(df[df.columns[1:]]))

In [8]:
plt.scatter(mcaFic.fs_r()[:, 0],mcaFic.fs_r()[:, 1])
for i, j, nom in zip(mcaFic.fs_r()[:, 0],mcaFic.fs_r()[:, 1], dc.index):
       plt.text(i, j, nom)
plt.show()

MemoryError: Unable to allocate 18.6 GiB for an array with shape (50001, 50001) and data type float64

In [73]:
df[df['revfyr']=='manquant'][df['montant_credit']=='?'][df['sitfam']=='NR']

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1112\806440614.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['revfyr']=='manquant'][df['montant_credit']=='?'][df['sitfam']=='NR']
C:\Users\lenovo\AppData\Local\Temp\ipykernel_1112\806440614.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['revfyr']=='manquant'][df['montant_credit']=='?'][df['sitfam']=='NR']


,num_cli,connexion_mois,source_web,typ_client,montant_credit,activite,anciennete,utilisation_credit,logmt,sitfam,dept,tr_age,revfyr,assurance
126,127,4,Liens Sponsorisés,Pas client ou ancien client,?,?,?,?,Autres,NR,Haute-Normandie,26-35 ans,manquant,?
256,257,3,Liens Sponsorisés,Pas client ou ancien client,?,?,?,?,Loge chez parents,NR,Rhône-Alpes,26-35 ans,manquant,?
668,669,7-10,Liens Sponsorisés,Pas client ou ancien client,?,?,?,?,Autres,NR,Haute-Normandie,26-35 ans,manquant,?
822,823,2,Accès Direct,Pas client ou ancien client,?,?,?,?,Autres,NR,Midi-Pyrenees,46-55 ans,manquant,?
1277,1278,2,Webmails,Pas client ou ancien client,?,?,?,?,Autres,NR,Centre,66-75 ans,manquant,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48709,48710,1,Accès Direct,Pas client ou ancien client,?,?,?,?,Autres,NR,Rhône-Alpes,26-35 ans,manquant,?
48780,48781,4,Liens Sponsorisés,Pas client ou ancien client,?,?,?,?,Autres,NR,ile-de-France,36-45 ans,manquant,?
48942,48943,7-10,Accès Direct,Pas client ou ancien client,?,?,?,?,Autres,NR,Rhône-Alpes,36-45 ans,manquant,?
48969,48970,5-6,Webmails,Pas client ou ancien client,?,?,?,?,Autres,NR,ile-de-France,36-45 ans,manquant,?


In [61]:
haha[5]

,count
activite,
actif,43290
?,3540
inactif,3170


In [28]:
df

,num_cli,connexion_mois,source_web,typ_client,montant_credit,activite,anciennete,utilisation_credit,logmt,sitfam,dept,tr_age,revfyr,assurance
0,1,3,Moteurs,Reserve d'argent,5.]6000e- 8000e],actif,>24 mois,+++,Locataire,Marie - Union libre,Lorraine,26-35 ans,2000 a 2999,credit assure
1,2,4,Liens Sponsorisés,Reserve d'argent,2.]1000e- 3000e],actif,>24 mois,+++,Locataire,Celibataire,Aquitaine,26-35 ans,1000 a 1499,credit assure
2,3,1,Moteurs,Reserve d'argent,4.]3000e- 6000e],actif,>24 mois,+++,Accession pro,Marie - Union libre,Lorraine,26-35 ans,2000 a 2999,credit non assure
3,4,4,Moteurs,Reserve d'argent,4.]3000e- 6000e],actif,6m-12m,+++,Locataire,Celibataire,Auvergne,18-25 ans,1500 a 1999,credit assure
4,5,5-6,Accès Direct,Reserve d'argent,6.]8000e- 12000e],actif,>24 mois,++,Loge chez employeur,Marie - Union libre,Haute-Normandie,36-45 ans,3000 a 3999,credit non assure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,49997,2,Accès Direct,Reserve d'argent,5.]6000e- 8000e],actif,>24 mois,+++,Loge chez parents,Celibataire,Poitou-Charentes,26-35 ans,1000 a 1499,credit assure
49997,49998,5-6,Moteurs,Reserve d'argent,6.]8000e- 12000e],actif,>24 mois,+++,Locataire,Divorce - separe,Provence-Alpes-Côte-Azur,56-65 ans,1500 a 1999,credit assure
49998,49999,3,Accès Direct,Credit voiture,4.]3000e- 6000e],actif,>24 mois,+++,Locataire,Marie - Union libre,Haute-Normandie,26-35 ans,3000 a 3999,credit non assure
49999,50000,1,Accès Direct,Reserve d'argent,4.]3000e- 6000e],inactif,>24 mois,-,Loge chez parents,Celibataire,ile-de-France,56-65 ans,1000 a 1499,credit assure


In [5]:
def supprime_accent(ligne):
    out = ""
    for mot in ligne:
        for c in range(len(mot)):
            if c!=len(mot):
                if mot[c:c+1] == 'é' or c == 'è' or c == 'ê' or c=='Ã©' or c=='Ã¨':
                    print('AAAAAAAAAAAAAA')
                    c = 'e'
                elif c == 'à':
                    c = 'a'
                elif c == 'ù' or c == 'û':
                    c = 'u'
                elif c == 'î':
                    c = 'i'
                elif c == 'ç':
                    c = 'c'
                out += c
    return out

In [11]:
def texte_sans_accent(txt):
    out=""
    for ligne in txt.split('\n'):
        out += supprime_accent(ligne) + '\n'
    return out

In [23]:
test.split('\n')[4][19:24]

'Ã©s;R'

In [21]:
len("2;4;Liens SponsorisÃ©")

21